In [1]:
# PARAMS:
IN_FILENAME = '../data_src/abcnews-date-text.csv'
OUT_FILENAME = '../data_clean/abcnews-date-text.csv'
# TL;DR change params & Kernel > Restart & Run All to "run this like a script"

In [2]:
!lsb_release -a
!pwd
!python -V
!which python
# --- EXPECT:
# No LSB modules are available.
# Distributor ID:	Ubuntu
# Description:	Ubuntu 18.04.1 LTS
# Release:	18.04
# Codename:	bionic
# /data/learn/fucap/data_cleanup
# Python 2.7.15 :: Anaconda, Inc.
# /home/neuronq/miniconda3/envs/fucap/bin/python

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.1 LTS
Release:	18.04
Codename:	bionic
/data/learn/fucap/data_cleanup
Python 2.7.15 :: Anaconda, Inc.
/home/neuronq/miniconda3/envs/fucap/bin/python


In [3]:
# autoreload
%load_ext autoreload
%autoreload 1

# write code that works in both Python 2 (2.7+) and 3 (3.5+)
from __future__ import absolute_import, division, print_function, unicode_literals
try:
    input = raw_input
    range = xrange
except NameError:
    pass

In [4]:
import os
import sys
from datetime import datetime
import datetime as dtm
import time
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pprint
pp = pprint.PrettyPrinter(indent=2).pprint
from IPython.display import display as dd
from IPython.display import Image

%matplotlib inline

# add project dir to import path
project_dir = os.path.split(os.getcwd())[0]
if project_dir not in sys.path:
    sys.path.append(project_dir)

%aimport lib.visualization
%aimport lib.helpers
from lib.visualization import simple_plot
import lib.helpers as helpers

np.set_printoptions(threshold=1000)  # 'nan' == all
pd.set_option('display.max_rows', 1000)

### Sources
https://www.kaggle.com/therohk/million-headlines/version/6

# /
---

In [8]:
def fill_missing(df, inplace=False):
    if not inplace:
        df = df.copy()

    df['missing'] = 0

    from_dt = df.index.values[0]
    to_dt = df.index.values[-1]
    delta = np.timedelta64(1, 'D')
    dt = to_dt - delta
    
    empty = [np.nan] * len(df.columns)
    empty[-1] = 1

    while dt > from_dt:
        if dt not in df.index:
            df.loc[dt] = empty
        dt -= delta
    
    df.sort_index(inplace=True)
    
    df.fillna(method='ffill', inplace=True)
    
    return df

In [9]:
data = pd.read_csv(IN_FILENAME)
print('> size:', len(data))
dd(data.dtypes)
dd(data.head())
dd(data.tail())

> size: 1103665


publish_date      int64
headline_text    object
dtype: object

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


,publish_date,headline_text
1103660,20171231,the ashes smiths warners near miss liven up bo...
1103661,20171231,timelapse: brisbanes new year fireworks
1103662,20171231,what 2017 meant to the kids of australia
1103663,20171231,what the papodopoulos meeting may mean for ausus
1103664,20171231,who is george papadopoulos the former trump ca...


In [10]:
data['datetime'] = pd.to_datetime(data['publish_date'].astype(str))
data.drop(['publish_date'], axis=1, inplace=True)
data.set_index('datetime', inplace=True)
dd(data.dtypes)
dd(data.head(2))

headline_text    object
dtype: object

,headline_text
datetime,
2003-02-19,aba decides against community broadcasting lic...
2003-02-19,act fire witnesses must be aware of defamation


In [12]:
data[data.index >= '2015-02-01'].shape

(173372, 1)

In [13]:
data.iloc[-1]

headline_text    who is george papadopoulos the former trump ca...
Name: 2017-12-31 00:00:00, dtype: object

In [7]:
patts = [
    r'\bbit[\s-]*coin',
    r'blocks?[\s-]*chain',
    r'(digital|electronic|\be|crypto)[\s-]*(currenc|coin|cash)'
    r'ethereum',
    r'\bbtc',
    r'\beth\b',
]
data = data[data['headline_text'].str.contains('|'.join(patts))]
print('> size after filter:', len(data))
dd(data.head())

/home/neuronq/miniconda3/envs/fucap/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


> size after filter: 79


,headline_text
datetime,
2011-05-14,race call btc cup doomben
2012-05-09,bufferings hopes slump after btc cup draw
2013-04-17,kohler value bitcoin gold
2013-05-11,cummings colt on song in btc cup
2013-10-26,us seizes bitcoins from man linked to silk road


In [13]:
data.to_csv(OUT_FILENAME)